In [3]:
import seaborn as sns
import hvplot.pandas
import pandas as pd
import panel as pn
filepath="/Users/Administrator/python practice/world_wide_COVID_data_daily_data.csv"
df = pd.read_csv(filepath)
df


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779
1,13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
2,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
3,11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266
4,10/12/2020,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.968658
...,...,...,...,...,...,...,...,...,...,...,...,...
61895,25/03/2020,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61896,24/03/2020,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61897,23/03/2020,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61898,22/03/2020,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN


In [56]:
df.fillna(0,inplace=True)#将Dataframe中空的数据替换成0
df = df.sort_values(['year','month','day'])#按照年月日对数据进行筛选排序

#筛选出Dataframe中各个不同的洲和国家，建立lists
continents_options = sorted(df['continentExp'].unique().tolist())
countries_options = sorted(df['countriesAndTerritories'].unique().tolist())

# 设置widgets
continents = pn.widgets.MultiSelect(value=['Asia'],name='Continents',options= continents_options, size=6)

countries = pn.widgets.MultiSelect(value=['China'],name='Countries', options= countries_options, size=8)

months = pn.widgets.IntRangeSlider(name='Months',start=1, end=12, value=(3,4), step=1)

cases_or_deaths = pn.widgets.RadioButtonGroup(name= "cases_or_deaths", options=['cases', 'deaths'])

#根据Widges中选择的值能绘成对应的图，且能根据选择的大洲实时更新对应的国家
@pn.depends(countries,months,cases_or_deaths)
def make_plot(countries,months,cases_or_deaths):
    df2 = df[df['countriesAndTerritories'].isin(countries)]
    df2 = df2[df2['month'].isin(range(months[0],months[1]))]
    plot = df2.hvplot.line(x = 'dateRep', y = cases_or_deaths, xlabel = 'Date',by = 'countriesAndTerritories',rot=90,width = 550,height = 300)
    return plot

@pn.depends(continents,watch = True)
def update_countries(continents):
    df1 = df[df['continentExp'].isin(continents)]
    countries.options = sorted(df1['countriesAndTerritories'].unique().tolist())


In [57]:
# pn.Row(pn.WidgetBox(continents,countries,months,cases_or_deaths,width=150),make_plot)


Row
    [0] WidgetBox(width=150)
        [0] MultiSelect(name='Continents', options=['Africa', 'America', ...], size=6, value=['Asia'])
        [1] MultiSelect(name='Countries', options=['Afghanistan', ...], size=8, value=['China'])
        [2] IntRangeSlider(end=12, name='Months', start=1, value=(3, 4), value_end=4, value_start=3)
        [3] RadioButtonGroup(name='cases_or_deaths', options=['cases', 'deaths'], value='cases')
    [1] ParamFunction(function)

In [52]:
#根据选定的国家绘制其相应月份新增和死亡病例的table
@pn.depends(countries,months)
def make_table(countries,months):
    df3 = df[df['countriesAndTerritories'].isin(countries)]
    df3 = df3[df3['month'].isin(range(months[0],months[1]))]    
    table = df3.hvplot.table(title = 'Data table' )
    return table


#设置template样式
template = pn.template.FastListTemplate(site="MED5018", title=" Worldwide COVID19 datum", sidebar=["## Settings", pn.WidgetBox(continents,countries,months,cases_or_deaths)],main=['Main plot',make_plot,make_table]).servable()
